## Aqui fazemos a primeira etapa do code, onde é feito o processamento dos dados baixados da ação correspondente, direto do site da Fundamentus. São explicados todos os passos do processamento. 

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import yfinance as yf
from fancyimpute import KNN


pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Using TensorFlow backend.


# DADOS FUNDAMENTALISTAS - FUNDAMENTUS

Ao fazer o download do site da Fundamentus, os dados vem em formato excel, então pegamos e convertemos facilmente para cvs.

Aqui trazemos o csv da ação para o notebook. Ele necessita do ".T" para que ele faça uma transposição, pois os atributos estavam nas linhas e as amostras nas colunas, assim invertemos isso, ficando no formato convencional utilizado.

Após, dropamos uma coluna inútil.

A coluna que mostra as datas, que estão separadas por trimestre, fica com um título NaN e está na primeira posição das colunas, assim precisamos atribuir o devido nome a coluna.

Assim, criamos uma coluna chamada "Data Trimeste", ao que lhe é atribuida as datas, e na sequência deletamos a coluna com o títlo NaN, através do iloc que engloba apenas colunas que tenham título "notnull"(naõ nulos).


A coluna "Data Trimestre" fica posicionada como última coluna do dataset, para ficar mais claro, apagamos, colocamos em uma variavel auxiliar, e a inserimos na primeira posição das colunas.

In [2]:
########################## COMEÇA O CÓDIGO QUE EXTRAI OS DADOS FUNDAMENTALISTAS ##########################

fundamental_data = pd.read_csv("vale3.csv",index_col=0, header=None).T
fundamental_data.drop(columns = 'XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data', inplace = True)


fundamental_data['Data Trimestre'] = fundamental_data.iloc[0:,0]
fundamental_data = fundamental_data.iloc[:, fundamental_data.columns.notnull()]

data = fundamental_data['Data Trimestre']
fundamental_data.drop(labels=['Data Trimestre'], axis=1,inplace = True)
fundamental_data.insert(0, 'Data Trimestre', data)


fundamental_data.head()

,Data Trimestre,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
1,30/06/2019,366.869.545,60.989.088,23.176.176,0,12.692.307,18.105.004,0,4.570.655,0,2.444.947,66.773.213,0,0,0,362.798,0,0,29.501.137,0,6.150.559,11.937.606,194.051.998,33.117.628,0,366.869.545,49.175.818,2.462.653,14.970.936,4.031.042,5.844.944,3.057.103,0,1.634.323,17.174.818,0,151.711.400,61.713.777,3.673.885,1.239.946,5.628.965,0,79.454.822,0,0,2.843.099,163.139.207,77.299.999,-2.714.714,0,39.664.894,-6.806.122,-6.667.409,62.362.563,0,0
2,31/03/2019,366.875.542,58.742.235,19.512.945,0,10.729.878,19.129.780,0,6.839.683,0,2.529.953,67.650.007,0,0,0,600.893,0,0,30.049.057,0,6.223.848,12.750.666,194.454.012,33.278.640,0,366.875.542,48.573.219,2.059.423,12.981.326,4.294.870,11.435.445,4.291.483,0,1.431.118,12.079.554,0,150.042.280,61.813.887,3.730.876,1.212.704,5.938.071,0,77.346.734,0,0,3.168.431,165.091.628,77.299.999,969.374,0,35.980.767,-6.421.966,-5.616.174,62.879.633,0,0
3,31/12/2018,341.714.862,59.256.115,22.412.548,0,11.669.973,17.215.949,0,5.525.300,0,2.432.348,51.631.833,0,0,0,589.178,0,0,26.767.477,0,6.247.965,12.495.002,187.481.506,30.850.392,0,341.714.862,35.516.666,4.054.028,13.609.716,3.331.427,3.888.522,4.392.479,0,2.183.871,4.056.619,0,132.516.004,56.039.432,3.721.696,1.334.669,5.935.772,0,65.484.427,0,0,3.279.552,170.402.628,77.299.999,969.374,0,35.896.254,0,-5.516.983,61.753.975,0,0
4,30/09/2018,342.278.570,60.585.832,24.424.372,0,11.150.082,16.238.472,0,6.382.340,0,2.390.565,47.987.573,0,0,0,618.832,0,0,22.874.839,0,6.425.454,12.598.114,189.916.692,31.190.331,0,342.278.570,36.718.207,3.517.680,16.168.571,3.162.779,5.497.638,2.017.100,0,1.584.239,4.770.200,0,138.866.803,61.808.202,3.838.802,1.971.298,6.851.634,0,64.396.857,0,0,3.435.501,163.258.051,77.299.999,1.020.142,0,19.852.984,3.426.400,-5.249.021,66.907.537,0,0
5,30/06/2018,339.647.169,59.639.296,24.557.353,0,10.350.142,15.417.622,0,6.478.245,0,2.835.935,48.945.807,0,0,0,593.466,0,0,25.199.423,0,6.077.764,12.440.545,187.816.280,30.805.232,0,339.647.169,35.420.455,2.674.345,13.832.036,3.450.536,7.027.064,2.103.283,0,1.310.411,5.022.780,0,138.888.872,62.015.840,3.712.720,2.382.543,6.471.644,0,64.306.115,0,0,3.622.700,161.715.158,77.299.999,1.020.142,0,21.792.299,5.367.437,-6.371.153,62.606.426,0,0


Aqui é o processo de manipulação das datas, muito importante na execução do code.

Primeiro utilizamos pd.to_datetime, função própria do Pandas para que ele entenda o formato e de que se trata de informações sobre data.

Subdividimos em colunas distintas, informações sobre "Dia", "Mês" e "Ano", logo a coluna "Data Trimestre" já não nos era mais útil.

Assim, novamente colocamos as três novas colunas em váriaveis auxiliares, para posiciona-las nas primeiras colunas do dataset.

In [3]:

fundamental_data['Data Trimestre'] = pd.to_datetime(fundamental_data['Data Trimestre'].str.strip(), format='%d/%m/%Y')


fundamental_data["Day"] = fundamental_data['Data Trimestre'].map(lambda x: x.day)
fundamental_data["Month"] = fundamental_data['Data Trimestre'].map(lambda x: x.month)
fundamental_data["Year"] = fundamental_data['Data Trimestre'].map(lambda x: x.year)

fundamental_data.drop(columns = 'Data Trimestre', inplace = True)

year = fundamental_data['Year']
fundamental_data.drop(labels=['Year'], axis=1,inplace = True)
fundamental_data.insert(0, 'Year', year)

month = fundamental_data['Month']
fundamental_data.drop(labels=['Month'], axis=1,inplace = True)
fundamental_data.insert(1, 'Month', month)

day = fundamental_data['Day']
fundamental_data.drop(labels=['Day'], axis=1,inplace = True)
fundamental_data.insert(2, 'Day', day)


fundamental_data.head()

,Year,Month,Day,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
1,2019,6,30,366.869.545,60.989.088,23.176.176,0,12.692.307,18.105.004,0,4.570.655,0,2.444.947,66.773.213,0,0,0,362.798,0,0,29.501.137,0,6.150.559,11.937.606,194.051.998,33.117.628,0,366.869.545,49.175.818,2.462.653,14.970.936,4.031.042,5.844.944,3.057.103,0,1.634.323,17.174.818,0,151.711.400,61.713.777,3.673.885,1.239.946,5.628.965,0,79.454.822,0,0,2.843.099,163.139.207,77.299.999,-2.714.714,0,39.664.894,-6.806.122,-6.667.409,62.362.563,0,0
2,2019,3,31,366.875.542,58.742.235,19.512.945,0,10.729.878,19.129.780,0,6.839.683,0,2.529.953,67.650.007,0,0,0,600.893,0,0,30.049.057,0,6.223.848,12.750.666,194.454.012,33.278.640,0,366.875.542,48.573.219,2.059.423,12.981.326,4.294.870,11.435.445,4.291.483,0,1.431.118,12.079.554,0,150.042.280,61.813.887,3.730.876,1.212.704,5.938.071,0,77.346.734,0,0,3.168.431,165.091.628,77.299.999,969.374,0,35.980.767,-6.421.966,-5.616.174,62.879.633,0,0
3,2018,12,31,341.714.862,59.256.115,22.412.548,0,11.669.973,17.215.949,0,5.525.300,0,2.432.348,51.631.833,0,0,0,589.178,0,0,26.767.477,0,6.247.965,12.495.002,187.481.506,30.850.392,0,341.714.862,35.516.666,4.054.028,13.609.716,3.331.427,3.888.522,4.392.479,0,2.183.871,4.056.619,0,132.516.004,56.039.432,3.721.696,1.334.669,5.935.772,0,65.484.427,0,0,3.279.552,170.402.628,77.299.999,969.374,0,35.896.254,0,-5.516.983,61.753.975,0,0
4,2018,9,30,342.278.570,60.585.832,24.424.372,0,11.150.082,16.238.472,0,6.382.340,0,2.390.565,47.987.573,0,0,0,618.832,0,0,22.874.839,0,6.425.454,12.598.114,189.916.692,31.190.331,0,342.278.570,36.718.207,3.517.680,16.168.571,3.162.779,5.497.638,2.017.100,0,1.584.239,4.770.200,0,138.866.803,61.808.202,3.838.802,1.971.298,6.851.634,0,64.396.857,0,0,3.435.501,163.258.051,77.299.999,1.020.142,0,19.852.984,3.426.400,-5.249.021,66.907.537,0,0
5,2018,6,30,339.647.169,59.639.296,24.557.353,0,10.350.142,15.417.622,0,6.478.245,0,2.835.935,48.945.807,0,0,0,593.466,0,0,25.199.423,0,6.077.764,12.440.545,187.816.280,30.805.232,0,339.647.169,35.420.455,2.674.345,13.832.036,3.450.536,7.027.064,2.103.283,0,1.310.411,5.022.780,0,138.888.872,62.015.840,3.712.720,2.382.543,6.471.644,0,64.306.115,0,0,3.622.700,161.715.158,77.299.999,1.020.142,0,21.792.299,5.367.437,-6.371.153,62.606.426,0,0


Essa é uma parte bem crítica, resetamos o índice(para que ele fique com a contagem correta de 0 até o total de linhas). E você deve estar se perguntando: por quê pegar o dataset a partir da 4 posição ?

Esse dataset está em formato descrescente com base nas datas, então, la no final do projeto, quando formos comparar os algortimos de classificação com a série temporal, para validação temos que comparar com dados que já ocorreram, par ter uma noção do desempenho.

Lembrando que se pegassemos de 0:, nesse caso, ele ia predizer dados a partir do 3T de 2018 até o 3T de 2019, sendo que o TCC estava sendo feito nessa época, então não tinhamos os dados reais necessarios para comparação, então por fins logisticos, reduzimos a ánalise um ano antes, para ao final do trabalho, quando compararmos os fundamentus e os algortimos de classificação com as séries temporais e a LSTM tenhamos tranquilidade para comparar com dados que ocorreram de fato para se ter uma boa base para inferir desempenho e tirar conclusões.

In [4]:
fundamental_data.reset_index(drop=True, inplace=True)
fundamental_data = fundamental_data[0:42]
fundamental_data.reset_index(drop=True, inplace=True)

fundamental_data.head()

,Year,Month,Day,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,2019,6,30,366.869.545,60.989.088,23.176.176,0,12.692.307,18.105.004,0,4.570.655,0,2.444.947,66.773.213,0,0,0,362.798,0,0,29.501.137,0,6.150.559,11.937.606,194.051.998,33.117.628,0,366.869.545,49.175.818,2.462.653,14.970.936,4.031.042,5.844.944,3.057.103,0,1.634.323,17.174.818,0,151.711.400,61.713.777,3.673.885,1.239.946,5.628.965,0,79.454.822,0,0,2.843.099,163.139.207,77.299.999,-2.714.714,0,39.664.894,-6.806.122,-6.667.409,62.362.563,0,0
1,2019,3,31,366.875.542,58.742.235,19.512.945,0,10.729.878,19.129.780,0,6.839.683,0,2.529.953,67.650.007,0,0,0,600.893,0,0,30.049.057,0,6.223.848,12.750.666,194.454.012,33.278.640,0,366.875.542,48.573.219,2.059.423,12.981.326,4.294.870,11.435.445,4.291.483,0,1.431.118,12.079.554,0,150.042.280,61.813.887,3.730.876,1.212.704,5.938.071,0,77.346.734,0,0,3.168.431,165.091.628,77.299.999,969.374,0,35.980.767,-6.421.966,-5.616.174,62.879.633,0,0
2,2018,12,31,341.714.862,59.256.115,22.412.548,0,11.669.973,17.215.949,0,5.525.300,0,2.432.348,51.631.833,0,0,0,589.178,0,0,26.767.477,0,6.247.965,12.495.002,187.481.506,30.850.392,0,341.714.862,35.516.666,4.054.028,13.609.716,3.331.427,3.888.522,4.392.479,0,2.183.871,4.056.619,0,132.516.004,56.039.432,3.721.696,1.334.669,5.935.772,0,65.484.427,0,0,3.279.552,170.402.628,77.299.999,969.374,0,35.896.254,0,-5.516.983,61.753.975,0,0
3,2018,9,30,342.278.570,60.585.832,24.424.372,0,11.150.082,16.238.472,0,6.382.340,0,2.390.565,47.987.573,0,0,0,618.832,0,0,22.874.839,0,6.425.454,12.598.114,189.916.692,31.190.331,0,342.278.570,36.718.207,3.517.680,16.168.571,3.162.779,5.497.638,2.017.100,0,1.584.239,4.770.200,0,138.866.803,61.808.202,3.838.802,1.971.298,6.851.634,0,64.396.857,0,0,3.435.501,163.258.051,77.299.999,1.020.142,0,19.852.984,3.426.400,-5.249.021,66.907.537,0,0
4,2018,6,30,339.647.169,59.639.296,24.557.353,0,10.350.142,15.417.622,0,6.478.245,0,2.835.935,48.945.807,0,0,0,593.466,0,0,25.199.423,0,6.077.764,12.440.545,187.816.280,30.805.232,0,339.647.169,35.420.455,2.674.345,13.832.036,3.450.536,7.027.064,2.103.283,0,1.310.411,5.022.780,0,138.888.872,62.015.840,3.712.720,2.382.543,6.471.644,0,64.306.115,0,0,3.622.700,161.715.158,77.299.999,1.020.142,0,21.792.299,5.367.437,-6.371.153,62.606.426,0,0


Para fins didáticos, criei uma coluna "Quarter" que indica qual trimestre corresponde aquela linha, e a posicionei na primeira posição das colunas.

In [5]:
fundamental_data['Quarter'] = '2Q2019','1Q2019', '4Q2018', '3Q2018', '2Q2018', '1Q2018', '4Q2017', '3Q2017', '2Q2017', '1Q2017', '4Q2016', '3Q2016', '2Q2016', '1Q2016', '4Q2015', '3Q2015', '2Q2015', '1Q2015','4Q2014', '3Q2014', '2Q2014', '1Q2014', '4Q2013', '3Q2013', '2Q2013', '1Q2013', '4Q2012', '3Q2012', '2Q2012', '1Q2012', '4Q2011', '3Q2011', '2Q2011', '1Q2011', '4Q2010'
quarter = fundamental_data['Quarter']
fundamental_data.drop(labels=['Quarter'], axis=1, inplace=True)
fundamental_data.insert(0, 'Quarter', quarter)

fundamental_data.head()

,Quarter,Year,Month,Day,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,2Q2019,2019,6,30,366.869.545,60.989.088,23.176.176,0,12.692.307,18.105.004,0,4.570.655,0,2.444.947,66.773.213,0,0,0,362.798,0,0,29.501.137,0,6.150.559,11.937.606,194.051.998,33.117.628,0,366.869.545,49.175.818,2.462.653,14.970.936,4.031.042,5.844.944,3.057.103,0,1.634.323,17.174.818,0,151.711.400,61.713.777,3.673.885,1.239.946,5.628.965,0,79.454.822,0,0,2.843.099,163.139.207,77.299.999,-2.714.714,0,39.664.894,-6.806.122,-6.667.409,62.362.563,0,0
1,1Q2019,2019,3,31,366.875.542,58.742.235,19.512.945,0,10.729.878,19.129.780,0,6.839.683,0,2.529.953,67.650.007,0,0,0,600.893,0,0,30.049.057,0,6.223.848,12.750.666,194.454.012,33.278.640,0,366.875.542,48.573.219,2.059.423,12.981.326,4.294.870,11.435.445,4.291.483,0,1.431.118,12.079.554,0,150.042.280,61.813.887,3.730.876,1.212.704,5.938.071,0,77.346.734,0,0,3.168.431,165.091.628,77.299.999,969.374,0,35.980.767,-6.421.966,-5.616.174,62.879.633,0,0
2,4Q2018,2018,12,31,341.714.862,59.256.115,22.412.548,0,11.669.973,17.215.949,0,5.525.300,0,2.432.348,51.631.833,0,0,0,589.178,0,0,26.767.477,0,6.247.965,12.495.002,187.481.506,30.850.392,0,341.714.862,35.516.666,4.054.028,13.609.716,3.331.427,3.888.522,4.392.479,0,2.183.871,4.056.619,0,132.516.004,56.039.432,3.721.696,1.334.669,5.935.772,0,65.484.427,0,0,3.279.552,170.402.628,77.299.999,969.374,0,35.896.254,0,-5.516.983,61.753.975,0,0
3,3Q2018,2018,9,30,342.278.570,60.585.832,24.424.372,0,11.150.082,16.238.472,0,6.382.340,0,2.390.565,47.987.573,0,0,0,618.832,0,0,22.874.839,0,6.425.454,12.598.114,189.916.692,31.190.331,0,342.278.570,36.718.207,3.517.680,16.168.571,3.162.779,5.497.638,2.017.100,0,1.584.239,4.770.200,0,138.866.803,61.808.202,3.838.802,1.971.298,6.851.634,0,64.396.857,0,0,3.435.501,163.258.051,77.299.999,1.020.142,0,19.852.984,3.426.400,-5.249.021,66.907.537,0,0
4,2Q2018,2018,6,30,339.647.169,59.639.296,24.557.353,0,10.350.142,15.417.622,0,6.478.245,0,2.835.935,48.945.807,0,0,0,593.466,0,0,25.199.423,0,6.077.764,12.440.545,187.816.280,30.805.232,0,339.647.169,35.420.455,2.674.345,13.832.036,3.450.536,7.027.064,2.103.283,0,1.310.411,5.022.780,0,138.888.872,62.015.840,3.712.720,2.382.543,6.471.644,0,64.306.115,0,0,3.622.700,161.715.158,77.299.999,1.020.142,0,21.792.299,5.367.437,-6.371.153,62.606.426,0,0


Renomeei a coluna "Month" para "Cluster_Month", pois como se trata de trimestre, ele agrupa ali três meses correspondentes áquele trimestre, então sabemos que colunas onde tem Cluster_Month 3, englona na verdade, os meses 1,2 e 3. E assim por diante com os demais.

A coluna dia não nos é mais útil aqui, então tiramos a coluna.

In [6]:
fundamental_data.rename(columns={'Month' : 'Cluster_Month'},inplace = True)
fundamental_data.drop('Day', axis=1, inplace=True)

fundamental_data.head()

,Quarter,Year,Cluster_Month,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,2Q2019,2019,6,366.869.545,60.989.088,23.176.176,0,12.692.307,18.105.004,0,4.570.655,0,2.444.947,66.773.213,0,0,0,362.798,0,0,29.501.137,0,6.150.559,11.937.606,194.051.998,33.117.628,0,366.869.545,49.175.818,2.462.653,14.970.936,4.031.042,5.844.944,3.057.103,0,1.634.323,17.174.818,0,151.711.400,61.713.777,3.673.885,1.239.946,5.628.965,0,79.454.822,0,0,2.843.099,163.139.207,77.299.999,-2.714.714,0,39.664.894,-6.806.122,-6.667.409,62.362.563,0,0
1,1Q2019,2019,3,366.875.542,58.742.235,19.512.945,0,10.729.878,19.129.780,0,6.839.683,0,2.529.953,67.650.007,0,0,0,600.893,0,0,30.049.057,0,6.223.848,12.750.666,194.454.012,33.278.640,0,366.875.542,48.573.219,2.059.423,12.981.326,4.294.870,11.435.445,4.291.483,0,1.431.118,12.079.554,0,150.042.280,61.813.887,3.730.876,1.212.704,5.938.071,0,77.346.734,0,0,3.168.431,165.091.628,77.299.999,969.374,0,35.980.767,-6.421.966,-5.616.174,62.879.633,0,0
2,4Q2018,2018,12,341.714.862,59.256.115,22.412.548,0,11.669.973,17.215.949,0,5.525.300,0,2.432.348,51.631.833,0,0,0,589.178,0,0,26.767.477,0,6.247.965,12.495.002,187.481.506,30.850.392,0,341.714.862,35.516.666,4.054.028,13.609.716,3.331.427,3.888.522,4.392.479,0,2.183.871,4.056.619,0,132.516.004,56.039.432,3.721.696,1.334.669,5.935.772,0,65.484.427,0,0,3.279.552,170.402.628,77.299.999,969.374,0,35.896.254,0,-5.516.983,61.753.975,0,0
3,3Q2018,2018,9,342.278.570,60.585.832,24.424.372,0,11.150.082,16.238.472,0,6.382.340,0,2.390.565,47.987.573,0,0,0,618.832,0,0,22.874.839,0,6.425.454,12.598.114,189.916.692,31.190.331,0,342.278.570,36.718.207,3.517.680,16.168.571,3.162.779,5.497.638,2.017.100,0,1.584.239,4.770.200,0,138.866.803,61.808.202,3.838.802,1.971.298,6.851.634,0,64.396.857,0,0,3.435.501,163.258.051,77.299.999,1.020.142,0,19.852.984,3.426.400,-5.249.021,66.907.537,0,0
4,2Q2018,2018,6,339.647.169,59.639.296,24.557.353,0,10.350.142,15.417.622,0,6.478.245,0,2.835.935,48.945.807,0,0,0,593.466,0,0,25.199.423,0,6.077.764,12.440.545,187.816.280,30.805.232,0,339.647.169,35.420.455,2.674.345,13.832.036,3.450.536,7.027.064,2.103.283,0,1.310.411,5.022.780,0,138.888.872,62.015.840,3.712.720,2.382.543,6.471.644,0,64.306.115,0,0,3.622.700,161.715.158,77.299.999,1.020.142,0,21.792.299,5.367.437,-6.371.153,62.606.426,0,0


Correção de processamento, os dados vieram com "pontos" o que inviabiliza a manipulação númerica que o pandas compreende, assim tivemos que remover os pontos e atribuir o formato float, anteriormente veio com formato string.

Assim podemos ver a disposição dos números em relação a tabela gerada pela célula abaixo e a que tinhamos antes.

In [7]:
for key in ['Ativo Total','Ativo Circulante','Caixa e Equivalentes de Caixa AC','Aplicações Financeiras AC','Contas a Receber AC','Estoques AC', 'Ativos Biológicos AC','Tributos a Recuperar AC','Despesas Antecipadas AC','Outros Ativos Circulantes AC','Ativo Realizavel Longo Prazo ','Aplicações Financeiras Avaliadas a Valor Justo ACLP','Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP','Contas a Receber ACLP','Estoques ACLP','Ativos Biológicos ACLP','Tributos Diferidos ACLP','Despesas Antecipadas ACLP','Créditos com Partes Relacionadas ACLP','Outros Ativos Não Circulantes ACLP','Investimentos','Imobilizado','Intangível','Diferido','Passivo Total','Passivo Circulante','Obrigações Sociais e Trabalhistas PC','Fornecedores PC','Obrigações Fiscais PC','Empréstimos e Financiamentos PC','Passivos com Partes Relacionadas PC','Dividendos e JCP a Pagar PC','Outros PC','Provisões PC', 'Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC', 'Passivo Não Circulante','Empréstimos e Financiamentos PNC','Passivos com Partes Relacionadas PNC','Outros PNC','Tributos Diferidos PNC','Adiantamento para Futuro Aumento Capital PNC','Provisões PNC','Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC','Lucros e Receitas a Apropriar PNC','Participação dos Acionistas Não Controladores','Patrimônio Líquido','Capital Social Realizado','Reservas de Capital','Reservas de Reavaliação','Reservas de Lucros','Lucros/Prejuízos Acumulados','Ajustes de Avaliação Patrimonial','Ajustes Acumulados de Conversão','Outros Resultados Abrangentes','Adiantamento para Futuro Aumento Capital']:  
    fundamental_data[key] = fundamental_data[key].map(lambda x: str(x).replace(".", ""))
    fundamental_data[key] = fundamental_data[key].astype(float)


fundamental_data.head()

,Quarter,Year,Cluster_Month,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,2Q2019,2019,6,366869545.0,60989088.0,23176176.0,0.0,12692307.0,18105004.0,0.0,4570655.0,0.0,2444947.0,66773213.0,0.0,0.0,0.0,362798.0,0.0,0.0,29501137.0,0.0,6150559.0,11937606.0,194051998.0,33117628.0,0.0,366869545.0,49175818.0,2462653.0,14970936.0,4031042.0,5844944.0,3057103.0,0.0,1634323.0,17174818.0,0.0,151711400.0,61713777.0,3673885.0,1239946.0,5628965.0,0.0,79454822.0,0.0,0.0,2843099.0,163139207.0,77299999.0,-2714714.0,0.0,39664894.0,-6806122.0,-6667409.0,62362563.0,0.0,0.0
1,1Q2019,2019,3,366875542.0,58742235.0,19512945.0,0.0,10729878.0,19129780.0,0.0,6839683.0,0.0,2529953.0,67650007.0,0.0,0.0,0.0,600893.0,0.0,0.0,30049057.0,0.0,6223848.0,12750666.0,194454012.0,33278640.0,0.0,366875542.0,48573219.0,2059423.0,12981326.0,4294870.0,11435445.0,4291483.0,0.0,1431118.0,12079554.0,0.0,150042280.0,61813887.0,3730876.0,1212704.0,5938071.0,0.0,77346734.0,0.0,0.0,3168431.0,165091628.0,77299999.0,969374.0,0.0,35980767.0,-6421966.0,-5616174.0,62879633.0,0.0,0.0
2,4Q2018,2018,12,341714862.0,59256115.0,22412548.0,0.0,11669973.0,17215949.0,0.0,5525300.0,0.0,2432348.0,51631833.0,0.0,0.0,0.0,589178.0,0.0,0.0,26767477.0,0.0,6247965.0,12495002.0,187481506.0,30850392.0,0.0,341714862.0,35516666.0,4054028.0,13609716.0,3331427.0,3888522.0,4392479.0,0.0,2183871.0,4056619.0,0.0,132516004.0,56039432.0,3721696.0,1334669.0,5935772.0,0.0,65484427.0,0.0,0.0,3279552.0,170402628.0,77299999.0,969374.0,0.0,35896254.0,0.0,-5516983.0,61753975.0,0.0,0.0
3,3Q2018,2018,9,342278570.0,60585832.0,24424372.0,0.0,11150082.0,16238472.0,0.0,6382340.0,0.0,2390565.0,47987573.0,0.0,0.0,0.0,618832.0,0.0,0.0,22874839.0,0.0,6425454.0,12598114.0,189916692.0,31190331.0,0.0,342278570.0,36718207.0,3517680.0,16168571.0,3162779.0,5497638.0,2017100.0,0.0,1584239.0,4770200.0,0.0,138866803.0,61808202.0,3838802.0,1971298.0,6851634.0,0.0,64396857.0,0.0,0.0,3435501.0,163258051.0,77299999.0,1020142.0,0.0,19852984.0,3426400.0,-5249021.0,66907537.0,0.0,0.0
4,2Q2018,2018,6,339647169.0,59639296.0,24557353.0,0.0,10350142.0,15417622.0,0.0,6478245.0,0.0,2835935.0,48945807.0,0.0,0.0,0.0,593466.0,0.0,0.0,25199423.0,0.0,6077764.0,12440545.0,187816280.0,30805232.0,0.0,339647169.0,35420455.0,2674345.0,13832036.0,3450536.0,7027064.0,2103283.0,0.0,1310411.0,5022780.0,0.0,138888872.0,62015840.0,3712720.0,2382543.0,6471644.0,0.0,64306115.0,0.0,0.0,3622700.0,161715158.0,77299999.0,1020142.0,0.0,21792299.0,5367437.0,-6371153.0,62606426.0,0.0,0.0


# DADOS DE PREÇO - API YAHOO FINANCE

Após determinandos os dados fundamentalistas da ação, chegou a hora de extrairmos os dados de preço da API aberta da Yahoo Finance.

Lembrando que temos que convertê-las ao formato de escala tempora de trimestre, para que possamos concatêna-las aos dados fundamentalistas, formando um dataset único da ação. Dados de preço e dados fundamentalistas referentes ao mesmo espaço temporal para cada amostra.

Abaixo, primeiro atribuimos a data de inicio e de fim que coletaremos os dados de preço, e depois trazemos o dataset com base nas datas, e 'm' nos diz respeito a escala temporal mês, como não há trimestre, tivemos que fazer a seguir a manipulação no código, com base nos dados originalmente mensais vindo da API.

In [8]:
####################### COMEÇA O CÓDIGO QUE EXTRAI DADOS DE PREÇO ###############################

start = datetime(2008, 12, 31)
end = datetime(2019, 5, 31)
price_data = web.get_data_yahoo('VALE3.SA', start, end, interval = 'm')

price_data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2009-01-01,34.290001,27.590000,27.690001,32.500000,86714600.0,21.311033
2009-02-01,39.709999,29.680000,31.809999,30.870001,109832800.0,20.242205
2009-03-01,33.669998,28.910000,29.950001,31.000000,94050600.0,20.327448
2009-04-01,36.599998,30.209999,30.400000,36.080002,78980900.0,23.658529
2009-05-01,41.340000,36.070000,36.080002,38.020000,91376300.0,25.300434


As datas na coluna "Date" estavam como índíce, e usamos o code abaixo para tornar "Date" uma coluna normal do dataset, já que vamos precisar manipula-la.

In [9]:
price_data["Date"] = price_data.index
price_data.reset_index(drop=True, inplace=True)


date = price_data['Date']
price_data.drop(labels=['Date'], axis=1,inplace = True)
price_data.insert(0, 'Date', date)

price_data.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2009-01-01,34.290001,27.590000,27.690001,32.500000,86714600.0,21.311033
1,2009-02-01,39.709999,29.680000,31.809999,30.870001,109832800.0,20.242205
2,2009-03-01,33.669998,28.910000,29.950001,31.000000,94050600.0,20.327448
3,2009-04-01,36.599998,30.209999,30.400000,36.080002,78980900.0,23.658529
4,2009-05-01,41.340000,36.070000,36.080002,38.020000,91376300.0,25.300434


Aqui começa a manipulação para transformarmos os dados de preço de escala mensal para escala trimestral.

Tal qual nos dados fundamentalistas, da coluna Date, criamos as colunas referentes a "dia", "mês" e "ano". Assim a coluna Date não nos é mais útil.

Manipulamos para colocar as três colunas referentes a data nas três primeiras posições das colunas no dataset.

In [10]:
price_data["Day"] = price_data['Date'].map(lambda x: x.day)
price_data["Month"] = price_data['Date'].map(lambda x: x.month)
price_data["Year"] = price_data['Date'].map(lambda x: x.year)

price_data.drop(columns = 'Date', inplace = True)



year = price_data['Year']
price_data.drop(labels=['Year'], axis=1,inplace = True)
price_data.insert(0, 'Year', year)

month = price_data['Month']
price_data.drop(labels=['Month'], axis=1,inplace = True)
price_data.insert(1, 'Month', month)

day = price_data['Day']
price_data.drop(labels=['Day'], axis=1,inplace = True)
price_data.insert(2, 'Day', day)

price_data.head()

,Year,Month,Day,High,Low,Open,Close,Volume,Adj Close
0,2009,1,1,34.290001,27.590000,27.690001,32.500000,86714600.0,21.311033
1,2009,2,1,39.709999,29.680000,31.809999,30.870001,109832800.0,20.242205
2,2009,3,1,33.669998,28.910000,29.950001,31.000000,94050600.0,20.327448
3,2009,4,1,36.599998,30.209999,30.400000,36.080002,78980900.0,23.658529
4,2009,5,1,41.340000,36.070000,36.080002,38.020000,91376300.0,25.300434


Nessa parte, o "for" percorre todas as linhas do dataset, e tal qual no padrão dos dados fundamentalistas, temos que atribuir aquele padrão que chamei de "Cluster_Month", onde o mês 3 representa o trimestre dos meses 1,2 e 3. Mês 6 o trimestre dos meses 4,5 e 6. E assim por diante.

Após, deletamos a coluna "dia", que não nos é útil.

In [11]:
for i,row in price_data.iterrows():   
    if (row['Month'] == 1) | (row['Month'] == 2) | (row['Month'] == 3):
        price_data.loc[i,'Month'] = 3
    
    elif (row['Month'] == 4) | (row['Month'] == 5) | (row['Month'] == 6):
         price_data.loc[i,'Month'] = 6
        
    elif (row['Month'] == 7) | (row['Month'] == 8) | (row['Month'] == 9):
         price_data.loc[i,'Month'] = 9
        
    else: 
          price_data.loc[i,'Month'] = 12



price_data.drop('Day', axis=1, inplace=True)

price_data.head()

,Year,Month,High,Low,Open,Close,Volume,Adj Close
0,2009,3,34.290001,27.590000,27.690001,32.500000,86714600.0,21.311033
1,2009,3,39.709999,29.680000,31.809999,30.870001,109832800.0,20.242205
2,2009,3,33.669998,28.910000,29.950001,31.000000,94050600.0,20.327448
3,2009,6,36.599998,30.209999,30.400000,36.080002,78980900.0,23.658529
4,2009,6,41.340000,36.070000,36.080002,38.020000,91376300.0,25.300434


E aqui tempos uma parte bem interessante. Agora agrupamos cada uma das três linhas correspondentes ao trimestre do ano correspondente.

Podemos olhar juntos para a primeira instrução abaixo e para a tabela acima para entender. Agrupamos por Ano e Mês, assim podemos ver as três primeiras linhas com "Month" igual a 3(na verdade são 1,2 e 3) que foram manipulados para facilitar esse processo. Então iremos agrupar essas três linhas, transformando em uma linha só correspondente ao primeiro trimestre de 2014 nesse caso. E assim convertemos os parâmetros de preço conforme suas especificidades.

- Abertura(Open): Utilização do método “first”, atribuindo o primeiro valor do trimestre.
   
- Fechamento(Close): Utilização do método “last”, atribuindo o último valor do trimestre.
      
- Máximo(High):Utilização do método “max”, atribuindo o máximo valor do trimestre.
 
- Mínimo(Low):Utilização do método “min”, atribuindo o mínimo valor do trimestre.

- Volume(Volume): Utilização do método “mean”, atribuindo a média dos valores do trimestre.

- Fechamento Ajustado(Adj Close):Utilização do método “mean”, atribuindo a média dos valores do trimestre.



Depois, novamente o processo de criar uma coluna "Quartes" com uma identificação mais intuitiva do trimestre e ano, e coloca-lo como a primeira posição das colunas.

Além de renomear a coluna "Month" para "Cluster_Month", tudo no mesmo padrão dos dados fundamentalistas.

In [12]:
final_data_price = price_data.groupby(['Year','Month']).agg({'High': 'max', 'Low': 'min', 'Open':'first', 'Close' : 'last', 'Volume':'mean', 'Adj Close':'mean'}).reset_index()


final_data_price['Quarter'] = '1Q2009', '2Q2009', '3Q2009', '4Q2009', '1Q2010', '2Q2010', '3Q2010', '4Q2010', '1Q2011', '2Q2011', '3Q2011', '4Q2011', '1Q2012', '2Q2012', '3Q2012','4Q2012', '1Q2013', '2Q2013', '3Q2013', '4Q2013', '1Q2014', '2Q2014', '3Q2014', '4Q2014', '1Q2015', '2Q2015', '3Q2015', '4Q2015', '1Q2016', '2Q2016', '3Q2016', '4Q2016', '1Q2017', '2Q2017', '3Q2017', '4Q2017', '1Q2018', '2Q2018', '3Q2018', '4Q2018','1Q2019','2Q2019'

quarter = final_data_price['Quarter']
final_data_price.drop(labels=['Quarter'], axis=1, inplace=True)
final_data_price.insert(0, 'Quarter', quarter)

final_data_price.rename(columns={'Month' : 'Cluster_Month'},inplace = True)

final_data_price.head()

,Quarter,Year,Cluster_Month,High,Low,Open,Close,Volume,Adj Close
0,1Q2009,2009,3,39.709999,27.590000,27.690001,31.000000,9.686600e+07,20.626895
1,2Q2009,2009,6,41.340000,30.209999,30.400000,34.500000,8.215820e+07,23.972338
2,3Q2009,2009,9,42.130001,31.540001,34.709999,41.189999,7.546623e+07,25.491198
3,4Q2009,2009,12,50.549999,39.410000,40.849998,49.660000,6.522510e+07,32.064050
4,1Q2010,2010,3,58.080002,45.700001,49.500000,57.150002,5.828830e+07,35.093661


# UNINDO DADOS DE PREÇO E DADOS FUNDAMENTALISTAS

Simplesmente fazendo um merge entre os dados de preço e fundamentalistas que tratamos até aqui.

A coluna "Year" e "Month" ficará repetida, então precisaremos renomear cada uma e excluir as outras duas que são a mesma coisa.

In [13]:
# MESCLA DOS DADOS DE PREÇO E OS FUNDAMENTALISTAS

dataset = pd.merge(final_data_price, fundamental_data, how='left', on='Quarter')

dataset.rename(columns={'Year_x' : 'Year'},inplace = True)
dataset.rename(columns={'Cluster_Month_x' : 'Cluster_Month'},inplace = True)

dataset.drop('Year_y', axis=1, inplace=True)

dataset.drop('Cluster_Month_y', axis=1, inplace=True)

dataset.head()

,Quarter,Year,Cluster_Month,High,Low,Open,Close,Volume,Adj Close,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Ativos Biológicos AC,Tributos a Recuperar AC,Despesas Antecipadas AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo ACLP,Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP,Contas a Receber ACLP,Estoques ACLP,Ativos Biológicos ACLP,Tributos Diferidos ACLP,Despesas Antecipadas ACLP,Créditos com Partes Relacionadas ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Passivos com Partes Relacionadas PC,Dividendos e JCP a Pagar PC,Outros PC,Provisões PC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Passivos com Partes Relacionadas PNC,Outros PNC,Tributos Diferidos PNC,Adiantamento para Futuro Aumento Capital PNC,Provisões PNC,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC,Lucros e Receitas a Apropriar PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,1Q2009,2009,3,39.709999,27.590000,27.690001,31.000000,9.686600e+07,20.626895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2Q2009,2009,6,41.340000,30.209999,30.400000,34.500000,8.215820e+07,23.972338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3Q2009,2009,9,42.130001,31.540001,34.709999,41.189999,7.546623e+07,25.491198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4Q2009,2009,12,50.549999,39.410000,40.849998,49.660000,6.522510e+07,32.064050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1Q2010,2010,3,58.080002,45.700001,49.500000,57.150002,5.828830e+07,35.093661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ao analisar o dataset, decidi excluir colunas que tinha mais de 1/3 dos dados faltantes, seguindo a lista abaixo de colunas "dropadas".

In [14]:
# exclui colunas que tem mais de 1/3 dos dados como zero ou faltante
dataset = dataset.drop('Ativos Biológicos AC',axis=1)
dataset = dataset.drop('Despesas Antecipadas AC',axis=1)
dataset = dataset.drop('Aplicações Financeiras Avaliadas a Valor Justo ACLP',axis=1)
dataset = dataset.drop('Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP',axis=1)
dataset = dataset.drop('Estoques ACLP',axis=1)
dataset = dataset.drop('Ativos Biológicos ACLP',axis=1)
dataset = dataset.drop('Despesas Antecipadas ACLP',axis=1)
dataset = dataset.drop('Créditos com Partes Relacionadas ACLP',axis=1)
dataset = dataset.drop('Diferido',axis=1)
dataset = dataset.drop('Passivos com Partes Relacionadas PC',axis=1)
dataset = dataset.drop('Dividendos e JCP a Pagar PC',axis=1)
dataset = dataset.drop('Provisões PC',axis=1)
dataset = dataset.drop('Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC',axis=1)
dataset = dataset.drop('Passivos com Partes Relacionadas PNC',axis=1)
dataset = dataset.drop('Adiantamento para Futuro Aumento Capital PNC',axis=1)
dataset = dataset.drop('Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC',axis=1)
dataset = dataset.drop('Lucros e Receitas a Apropriar PNC',axis=1)
dataset = dataset.drop('Reservas de Reavaliação',axis=1)
dataset = dataset.drop('Ajustes de Avaliação Patrimonial',axis=1)
dataset = dataset.drop('Ajustes Acumulados de Conversão',axis=1)
dataset = dataset.drop('Outros Resultados Abrangentes',axis=1)
dataset = dataset.drop('Adiantamento para Futuro Aumento Capital',axis=1)

# APLICAÇÃO DO ALGORTIMO KNN PARA PREENCHIMENTO DOS DADOS FALTANTES

Abaixo, separei apenas as colunas que serão imputadas para treinamento, tirando as de data e os id's. Assim ao invés de um método tradicional como preencher dados faltantes com média ou mediana, utilizei a metodologia com a biblioteca fancy impute, para aplicar KNN e preencher os dados faltantes com esse algortimo.

Na célula abaixo a separação.

Após trocar os valores 0 para "NaN", então realizamos o preenchimento de fato com o KNN.

In [15]:
#separa as colunas que serão usadas para treinamento em 'dataset'
pricing = dataset.iloc[0:,0:3]
dataset = dataset.iloc[0:,3:]
dataset.to_numpy()

X = dataset.replace(0, np.nan, inplace=True)
dataset.replace(0, np.nan, inplace=True)
final_data = KNN(k=3).fit_transform(dataset)

# ela tira as labels das colunas e aqui as coloco de volta
dataset = pd.DataFrame(data = final_data, columns = ['High','Low','Open','Close','Volume','Adj Close','Ativo Total','Ativo Circulante','Caixa e Equivalentes de Caixa AC','Aplicações Financeiras AC','Contas a Receber AC','Estoques AC','Tributos a Recuperar AC','Outros Ativos Circulantes AC','Ativo Realizavel Longo Prazo','Contas a Receber ACLP','Tributos Diferidos ACLP','Outros Ativos Não Circulantes ACLP','Investimentos','Imobilizado','Intangível','Passivo Total','Passivo Circulante','Obrigações Sociais e Trabalhistas PC','Fornecedores PC','Obrigações Fiscais PC','Empréstimos e Financiamentos PC','Outros PC','Passivo Não Circulante','Empréstimos e Financiamentos PNC','Outros PNC','Tributos Diferidos PNC','Provisões PNC','Participação dos Acionistas Não Controladores','Patrimônio Líquido','Capital Social Realizado','Reservas de Capital','Reservas de Lucros','Lucros/Prejuízos Acumulados'])
dataset = pd.concat([pricing, dataset], axis=1)

inference = dataset.iloc[-1,0:].to_frame().T
inference

Imputing row 1/42 with 33 missing, elapsed time: 0.002


,Quarter,Year,Cluster_Month,High,Low,Open,Close,Volume,Adj Close,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Tributos a Recuperar AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Contas a Receber ACLP,Tributos Diferidos ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Outros PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Outros PNC,Tributos Diferidos PNC,Provisões PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Lucros,Lucros/Prejuízos Acumulados
41,2Q2019,2019,6,53.8,45.82,52.35,49,4.01246e+08,48.2709,3.6687e+08,6.09891e+07,2.31762e+07,1.38728e+06,1.26923e+07,1.8105e+07,4.57066e+06,2.44495e+06,6.67732e+07,496591,2.12836e+07,6.15056e+06,1.19376e+07,1.94052e+08,3.31176e+07,3.6687e+08,4.91758e+07,2.46265e+06,1.49709e+07,4.03104e+06,5.84494e+06,1.63432e+06,1.51711e+08,6.17138e+07,1.23995e+06,5.62896e+06,7.94548e+07,2.8431e+06,1.63139e+08,7.73e+07,-2.71471e+06,3.96649e+07,-6.80612e+06


In [16]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 0 to 41
Data columns (total 42 columns):
Quarter                                          42 non-null object
Year                                             42 non-null int64
Cluster_Month                                    42 non-null int64
High                                             42 non-null float64
Low                                              42 non-null float64
Open                                             42 non-null float64
Close                                            42 non-null float64
Volume                                           42 non-null float64
Adj Close                                        42 non-null float64
Ativo Total                                      42 non-null float64
Ativo Circulante                                 42 non-null float64
Caixa e Equivalentes de Caixa AC                 42 non-null float64
Aplicações Financeiras AC                        42 non-null float64
Conta

# AQUI FINALMENTE PRODUZIMO A NOSSA AMOSTRA QUE SERÁ IMPUTADA AO NOSSO MODELO, SIMULANDO UMA SITUAÇÃO REAL. AO IMPUTAR OS DADOS DE PREÇO E DE FUNDAMENTOS CONFORME OS USADOS NO MODELO, QUAL STATUS DO TARGET ELE RETORNARÁ FRENTE A UMA POSSIVEL VALORIZAÇÃO ?

In [17]:
inference = inference.iloc[0:,3:]
inference

,High,Low,Open,Close,Volume,Adj Close,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,Aplicações Financeiras AC,Contas a Receber AC,Estoques AC,Tributos a Recuperar AC,Outros Ativos Circulantes AC,Ativo Realizavel Longo Prazo,Contas a Receber ACLP,Tributos Diferidos ACLP,Outros Ativos Não Circulantes ACLP,Investimentos,Imobilizado,Intangível,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas PC,Fornecedores PC,Obrigações Fiscais PC,Empréstimos e Financiamentos PC,Outros PC,Passivo Não Circulante,Empréstimos e Financiamentos PNC,Outros PNC,Tributos Diferidos PNC,Provisões PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Lucros,Lucros/Prejuízos Acumulados
41,53.8,45.82,52.35,49,4.01246e+08,48.2709,3.6687e+08,6.09891e+07,2.31762e+07,1.38728e+06,1.26923e+07,1.8105e+07,4.57066e+06,2.44495e+06,6.67732e+07,496591,2.12836e+07,6.15056e+06,1.19376e+07,1.94052e+08,3.31176e+07,3.6687e+08,4.91758e+07,2.46265e+06,1.49709e+07,4.03104e+06,5.84494e+06,1.63432e+06,1.51711e+08,6.17138e+07,1.23995e+06,5.62896e+06,7.94548e+07,2.8431e+06,1.63139e+08,7.73e+07,-2.71471e+06,3.96649e+07,-6.80612e+06


In [18]:
inference.to_csv("ROW_INFERENCE_VALE3.csv", index= False)